In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
users = pd.read_csv('C:\\Users\\panag\\Desktop\\Project Future\\Project\\data\\users.csv')

In [3]:
notif = pd.read_csv('C:\\Users\\panag\\Desktop\\Project Future\\Project\\data\\notifications.csv')

In [4]:
devices = pd.read_csv('C:\\Users\\panag\\Desktop\\Project Future\\Project\\data\\devices.csv')

In [5]:
trans1 = pd.read_csv('C:\\Users\\panag\\Desktop\\Project Future\\Project\\data\\transactions_1.csv')

In [6]:
trans2 = pd.read_csv('C:\\Users\\panag\\Desktop\\Project Future\\Project\\data\\transactions_2.csv')

In [7]:
trans3 = pd.read_csv('C:\\Users\\panag\\Desktop\\Project Future\\Project\\data\\transactions_3.csv')

In [8]:
transactions = pd.concat([trans1,trans2,trans3])

# 2.Preprocessing

## 2.1 Users Table

### 2.1.1 Change plan to paid or standard in users

In [9]:
def sub_plan(x):
    if x == 'STANDARD':
        return 0
    elif x == 'SILVER' or x == 'GOLD':
            return 1

In [10]:
users['plan'] = users['plan'].apply(sub_plan)
users['plan'].value_counts()

0    14394
1     1150
Name: plan, dtype: int64

In [11]:
users.sample(5)

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals
12955,user_12955,1986,DE,Berlin,2018-08-02 01:14:59.090716,0,0,1.0,0.0,3,0,0
13534,user_13534,1993,GB,London,2018-01-07 00:11:33.602974,0,0,NaN,NaN,12,0,0
5991,user_5991,1970,GB,South Shields,2018-06-06 10:57:41.612299,0,0,1.0,1.0,0,0,0
7228,user_7228,1997,IE,Waterford,2018-11-30 20:11:13.472336,0,0,1.0,1.0,0,0,0
8616,user_8616,1991,HU,Soltvadkert,2018-01-22 11:50:09.944972,0,0,NaN,NaN,0,0,0


### 2.1.2 Drop referrals from users_new

In [12]:
users= users.drop(['num_referrals','num_successful_referrals'], axis=1)
users

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts
0,user_0,1992,ES,Madrid,2018-09-10 18:46:42.843855,0,0,1.0,1.0,3
1,user_1,1985,LT,Klaipeda,2018-01-11 18:53:10.531146,1,0,NaN,NaN,75
2,user_2,1996,IE,Carrickmacross,2018-11-29 17:29:16.879324,0,0,NaN,NaN,71
3,user_3,1981,IE,Dublin,2018-02-17 11:58:08.319664,0,0,NaN,NaN,219
4,user_4,1992,GB,London,2018-01-08 18:40:21.097856,0,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
15539,user_15539,1997,PL,Bralin,2018-08-05 05:56:40.787458,0,0,1.0,1.0,0
15540,user_15540,1985,PL,Poznań,2018-03-29 12:19:05.567779,0,0,NaN,NaN,0
15541,user_15541,1978,GB,London,2018-07-13 15:56:50.339746,0,0,1.0,1.0,16
15542,user_15542,1977,IE,Dublin,2018-01-07 15:53:43.719983,0,1,1.0,1.0,44


### 2.1.3 Fixing city in users

In [13]:
def city(y):
    if y in ('1050 Wien','Vienna') :
        return 'Wien'
    elif y in ('1000 Bruxelles','Bruselas', 'Brussels Molenbeek', 'Bruxelles','bruxelles','1030'):
        return 'Brussels'
    elif y == 'Sofiya':
        return 'Sofia'
    elif y in ('nicosia','NICOSIA', 'Nicosia/Strovolos','Nicosoa'):
        return 'Nicosia'
    elif y in ('Prag','Praga', 'Prague 10', 'Prague 5', 'Prague 6','Prague 8','Prague 9','Praha','PRAHA','Praha 1','Praha 10',\
               'Praha 11','Praha 2','Praha 3','Praha 3-Vinohrady','Praha 5','Praha 5 - Zbraslav','Praha 6','Praha 7','Praha 8',\
               'Praha 8 Liben', 'Praha 9'):
        return 'Prague' 
    elif y == '10967/Berlin':
        return 'Berlin'
    elif y in ('Copenaghen','Copenhaga','copenhagen','Copenhagen S'):
        return 'Copenhagen'
    elif y == 'TALLINN' :
        return 'Tallinn'
    elif y in ('MAdrid','madrid'):
        return 'Madrid' 
    elif y in ('PARIS','paris','Paris 13e','75011','75018'):
        return 'Paris'
    elif y in ('30 Belmont Hall Court Belmont Grove, London, London' ,'Deptford London', 'Putney, London'):
        return 'London'
    elif y == 'gibraltar':
        return 'Gibraltar'
    elif y in ('ATHENS','Athens Plaka','Athens/Argyroupoli', 'Athina','CHALANDRI','Ekali', 'Galatsi','Galatsi Athens',\
               'Glyfada','Glyka Nera','Halandri','Holargos','Ilion','Kallithea','Kallithea,Athens','Khalándrion, Athens',\
               'Kifissia','Likovrisi','Marousi','Maroussi','Metamorfosis','Nikaia','Nikea Attikis','P.Faliro','Palaio Psihiko',\
               'Palio Faliro','Pefki','Vrilissia','ZOGRAFOS ATTIKIS','Agia Paraskevi','Agioi anargiroi'):
        return 'Athens' 
    elif y in ('Co Dublin','Co. Dublin','Co.Dublin','DUBLIN','duplin','Dublin 1','Dublin 11','Dublin 12','Dublin 13',\
               'Dublin 14','Dublin 15','Dublin 16','Dublin 18','Dublin 2','Dublin 22','DUBLIN 24','Dublin 3','Dublin 4',\
               'Dublin 5','Dublin 6','Dublin 6 west','Dublin 6w','Dublin 7','Dublin 8','Dublin 9','Dublin7','Dubljn','dublom'):
        return 'Dublin'
    elif y == 'Reykjavík'in y:
        return 'Reykjavik'
    elif y == 'Roma':
        return 'Rome'
    elif y in ('St Helier Jersey','St Hellier Jersey','St. Helier'):
        return 'St Helier' 
    elif y in ('Vilniaus r', 'Vilniaus r.','Vilniaus Raj, Rokantiškės', 'Vilniaus raj.','Vilniaus Rajonas','Vilnius','vilnius',\
               'VILNIUS') :
        return 'Vilnius'
    elif y in ('Rīga','Rigas rajons') :
        return 'Riga'
    elif y == 'Vallett':
        return 'Valletta'
    elif y == 'Amsterda' :
        return 'Amsterdam'
    elif y in ('Warszawa','Warszawa 79') :
        return 'Warsaw'
    elif y in ('Lisbao','Lisboa','lisbonne') :
        return 'Lisbon'
    elif y in ('BUCHAREST','Bucharest, RO','Bucharesy','Bucuresti','bucuresti','București'):
        return 'Bucharest'
    elif y == 'Ljubljana - Smartno':
        return 'Ljubljana'
    else:
        return y

In [14]:
users['city'] = users['city'].apply(city)

In [15]:
users.sample(10)

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts
2311,user_2311,1992,GB,Brighton,2018-11-10 03:44:46.052586,0,0,1.0,1.0,0
8847,user_8847,1995,GB,Ballynahinch,2018-07-03 22:44:55.218462,0,0,NaN,NaN,8
14048,user_14048,1973,GR,Athens,2018-07-03 18:23:27.813971,1,1,1.0,1.0,2
15365,user_15365,1991,RO,Bucharest,2018-07-05 20:24:56.538681,1,0,1.0,1.0,47
7300,user_7300,1982,GI,Gibraltar,2018-07-27 07:01:51.744134,0,0,1.0,1.0,98
1110,user_1110,1982,GB,London,2018-08-19 16:56:43.008372,0,0,NaN,NaN,0
2461,user_2461,1956,GB,London,2018-03-08 18:49:16.981357,0,0,NaN,NaN,2
9678,user_9678,1990,GB,Bangor,2018-06-09 13:50:44.820754,1,0,0.0,1.0,16
7249,user_7249,1979,ES,VALENCIA,2018-10-24 17:06:41.385461,0,0,1.0,1.0,0
9308,user_9308,1989,PL,Nieporęt,2018-12-17 22:24:44.664780,0,0,NaN,NaN,11


#### 2.1.3.1 Group cities as capital or not

In [16]:
def capital(x):
    if x in ('Wien','Canberra','Brussels','Sofia','Bern','Nicosia','Prague','Berlin','Copenhagen','Tallinn','Madrid',\
             'Helsinki','Paris','London','St Peter Port','Gibraltar', 'Basse-Terre','Athens','Zagreb','Budabest','Dublin',\
             'Douglas','Reykjavik','Rome','St Helier','Vaduz','Vilnius','Luxembourg','Riga','Fort de France','Valletta',\
             'Amsterdam','Oslo','Warsaw','Lisbon','St Denis','Bucharest','Stockholm','Ljubjana','Bratislava'):
        return 1
    else:
        return 0

In [17]:
users['city'] = users['city'].apply(capital)

In [18]:
pd.crosstab(index=users['city'], columns=users['plan'])

plan,0,1
city,,
0,10645,810
1,3749,340


In [19]:
users['city'].value_counts()

0    11455
1     4089
Name: city, dtype: int64

#### 2.1.3.2 Years

In [20]:
#split birth_year into yeargroups

years = pd.DataFrame(users, columns=['birth_year'])

bins = [1929, 1934, 1939, 1944, 1949, 1954, 1959, 1964, 1969, 1974, 1979, 1984, 1989, 1994, 1999, 2001]
labels = ['1929-1933', '1934-1938', '1939-1943', '1944-1948', '1949-1953','1954-1958','1959-1963', '1964-1968', '1969-1973', 
         '1974-1978', '1979-1983', '1984-1988', '1989-1993', '1994-1998', '1999-2003']
years['year_range'] = pd.cut(years.birth_year, bins, labels = labels,include_lowest = True)

In [21]:
mappings = {'1929-1933': 1, '1934-1938' :2, '1939-1943':3, '1944-1948':4, '1949-1953':5,'1954-1958':6,'1959-1963':7, 
            '1964-1968':8, '1969-1973':9, '1974-1978':10, '1979-1983':11, '1984-1988':12, '1989-1993':13, '1994-1998':14,
            '1999-2003': 15}

users['birth_year'] = years['year_range'].apply(lambda x: mappings[x])

In [22]:
users['birth_year'].value_counts()

13    3392
12    3173
14    2125
11    2116
10    1485
9     1072
8      795
7      534
6      324
15     197
5      194
4       93
3       32
2        7
1        5
Name: birth_year, dtype: int64

#### 2.1.3.3 Created Date

In [23]:
cr_date=[]
for i in users['created_date']:    
    cr_date.append(i[:7])

In [24]:
cr_date_df = pd.DataFrame({'created_date':cr_date})
cr_date_df['created_date'].unique()

array(['2018-09', '2018-01', '2018-11', '2018-02', '2018-08', '2018-07',
       '2018-03', '2018-12', '2018-04', '2018-10', '2018-05', '2018-06',
       '2019-01'], dtype=object)

In [25]:
mappings = {'2018-01': 1, '2018-02' :1, '2018-03':2, '2018-04':2, '2018-05':2,'2018-06':2,'2018-07':2,'2018-08':2, '2018-09':1,
            '2018-10':1, '2018-11':1, '2018-12':1, '2019-01':1}
users['created_date'] = cr_date_df['created_date'].apply(lambda x: mappings[x])

In [26]:
pd.crosstab(index=users['created_date'], columns=users['plan'])

plan,0,1
created_date,,
1,7440,543
2,6954,607


In [27]:
users.head(2)

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts
0,user_0,13,ES,1,1,0,0,1.0,1.0,3
1,user_1,12,LT,0,1,1,0,NaN,NaN,75


#### 2.1.3.4 Regions (country)

In [28]:
def regions(x):
    if x in ('DK','EE','FI','GG','IE','IM','IS','JE','LT','LV','NO','SE','AT','CH','CZ','DE','HU','LI', 'PL','SI','SK'):
        return 'North-East'
    elif x in ('BE','FR','LU','NL','BG','HR' ,'RO','CY', 'SP', 'GI', 'GR', 'IT', 'MT', 'PT','AU', 'GP', 'MQ', 'RE'):
        return 'West-South'
    else:
        return 'UK'

In [29]:
users['country'] = users['country'].apply(regions)

In [30]:
users['country'].value_counts()

UK            5802
North-East    5221
West-South    4521
Name: country, dtype: int64

In [31]:
country_enc= pd.get_dummies(users['country'])
users = pd.concat([users,country_enc],axis=1)
users= users.drop(['country'], axis=1)

In [32]:
users= users.drop(['attributes_notifications_marketing_push', 'attributes_notifications_marketing_email'], axis=1)

In [33]:
users.head(2)

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South
0,user_0,13,1,1,0,0,3,0,1,0
1,user_1,12,0,1,1,0,75,1,0,0


## Describe Num_contacts- amound_usd

In [34]:
users['num_contacts'].describe()  # Range [0,2918]

count    15544.000000
mean        17.874228
std         47.549929
min          0.000000
25%          0.000000
50%          5.000000
75%         19.000000
max       2918.000000
Name: num_contacts, dtype: float64

In [35]:
transactions['amount_usd'].describe()  # Range [0, 74.641.551.593]

count    2.181026e+06
mean     1.189225e+05
std      6.955454e+07
min      0.000000e+00
25%      2.030000e+00
50%      8.510000e+00
75%      2.807000e+01
max      7.464155e+10
Name: amount_usd, dtype: float64

# 2.2 Devices

In [36]:
def brand(x):
    if x in ('Android', 'Unknown'):
        return 0
    else:
        return 1

In [37]:
devices['brand'] = devices['brand'].apply(brand)

In [38]:
users = pd.merge(users,devices,on='user_id',how='inner')

In [39]:
users

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,brand
0,user_0,13,1,1,0,0,3,0,1,0,1
1,user_1,12,0,1,1,0,75,1,0,0,1
2,user_2,14,0,1,0,0,71,1,0,0,1
3,user_3,11,1,1,0,0,219,1,0,0,1
4,user_4,13,1,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
15539,user_15539,14,0,2,0,0,0,1,0,0,0
15540,user_15540,12,0,2,0,0,0,1,0,0,1
15541,user_15541,10,1,2,0,0,16,0,1,0,0
15542,user_15542,10,1,1,0,1,44,1,0,0,1


## 2.3 Transactions Table

In [40]:
users_trans = pd.merge(users,transactions,on='user_id',how='inner')

# 2.3.1  Transactions_type

In [41]:
pd.crosstab(index=transactions['transactions_type'], columns=transactions['user_id'])

user_id,user_0,user_1,user_10,user_100,user_1000,user_10000,user_10001,user_10002,user_10003,user_10004,...,user_9990,user_9991,user_9992,user_9993,user_9994,user_9995,user_9996,user_9997,user_9998,user_9999
transactions_type,,,,,,,,,,,,,,,,,,,,,
ATM,0,5,3,6,1,15,0,0,0,0,...,0,0,1,0,3,0,1,5,3,0
CARD_PAYMENT,343,86,61,43,20,25,42,6,16,0,...,2,0,45,208,62,4,25,54,12,0
CARD_REFUND,1,0,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
CASHBACK,0,0,0,0,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
EXCHANGE,5,0,3,0,0,16,5,1,0,1,...,0,0,5,0,2,1,5,20,7,0
FEE,0,0,0,0,2,1,0,0,1,0,...,0,1,1,1,0,0,1,1,0,0
REFUND,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TAX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TOPUP,173,27,44,17,7,28,9,3,3,2,...,3,1,13,19,16,3,5,41,10,10


In [42]:
numb_trans= pd.crosstab(index=transactions['transactions_type'], columns=transactions['user_id'])

In [43]:
numb_trans.sum()

user_id
user_0       545
user_1       146
user_10      143
user_100      66
user_1000     47
            ... 
user_9995      8
user_9996     37
user_9997    146
user_9998     36
user_9999     10
Length: 15012, dtype: int64

In [44]:
number_of_transactions = pd.DataFrame({'number_of_transactions': numb_trans.sum()})

In [45]:
users =pd.merge(users,number_of_transactions, how='left',left_on='user_id',right_index=True)

In [46]:
users

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,brand,number_of_transactions
0,user_0,13,1,1,0,0,3,0,1,0,1,545.0
1,user_1,12,0,1,1,0,75,1,0,0,1,146.0
2,user_2,14,0,1,0,0,71,1,0,0,1,282.0
3,user_3,11,1,1,0,0,219,1,0,0,1,158.0
4,user_4,13,1,1,0,0,0,0,1,0,1,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15539,user_15539,14,0,2,0,0,0,1,0,0,0,134.0
15540,user_15540,12,0,2,0,0,0,1,0,0,1,16.0
15541,user_15541,10,1,2,0,0,16,0,1,0,0,1115.0
15542,user_15542,10,1,1,0,1,44,1,0,0,1,30.0


# 2.3.2 Transactions State

In [47]:
pd.crosstab(index=transactions['transactions_state'], columns=transactions['user_id'])

user_id,user_0,user_1,user_10,user_100,user_1000,user_10000,user_10001,user_10002,user_10003,user_10004,...,user_9990,user_9991,user_9992,user_9993,user_9994,user_9995,user_9996,user_9997,user_9998,user_9999
transactions_state,,,,,,,,,,,,,,,,,,,,,
CANCELLED,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
COMPLETED,520,133,123,55,43,78,51,5,21,2,...,4,4,60,219,74,4,31,128,30,1
DECLINED,6,6,16,4,4,6,3,2,8,0,...,0,0,4,6,6,1,5,5,5,0
FAILED,9,1,0,4,0,6,0,0,0,0,...,0,0,2,0,0,1,0,5,0,6
PENDING,2,0,1,0,0,4,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
REVERTED,8,6,3,3,0,3,2,3,1,1,...,1,0,2,9,4,2,1,8,1,3


In [48]:
completed_transactions =pd.crosstab(index=transactions['transactions_state'], columns=transactions['user_id']).iloc[1,:]

In [49]:
completed_transactions

user_id
user_0       520
user_1       133
user_10      123
user_100      55
user_1000     43
            ... 
user_9995      4
user_9996     31
user_9997    128
user_9998     30
user_9999      1
Name: COMPLETED, Length: 15012, dtype: int64

In [50]:
completed_transactions = pd.DataFrame({'completed_transactions': completed_transactions})

In [51]:
completed_transactions

,completed_transactions
user_id,
user_0,520
user_1,133
user_10,123
user_100,55
user_1000,43
...,...
user_9995,4
user_9996,31
user_9997,128


In [52]:
users =pd.merge(users,completed_transactions, how='left',left_on='user_id',right_index=True)

In [53]:
users.head(2)

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,brand,number_of_transactions,completed_transactions
0,user_0,13,1,1,0,0,3,0,1,0,1,545.0,520.0
1,user_1,12,0,1,1,0,75,1,0,0,1,146.0,133.0


# 2.3.3 Created Date

In [54]:
transactions['created_date'].max()

'2019-05-16 18:22:16.959896'

In [55]:
transactions

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_4368,2018-04-03 03:34:21.784487
1,transaction_1,CARD_PAYMENT,AED,15.50,COMPLETED,FALSE,4111.0,Dubai,ARE,OUTBOUND,user_2355,2019-03-19 06:15:59.537032
2,transaction_2,CARD_PAYMENT,AED,43.40,COMPLETED,FALSE,5814.0,Dubai,ARE,OUTBOUND,user_2355,2019-03-18 18:53:41.323032
3,transaction_3,TRANSFER,AED,10043.01,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_2355,2019-03-22 14:20:01.513032
4,transaction_4,CARD_PAYMENT,AED,43.81,COMPLETED,FALSE,5651.0,Abu Dhabi,ARE,OUTBOUND,user_14318,2019-03-22 05:42:50.316652
...,...,...,...,...,...,...,...,...,...,...,...,...
681021,transaction_2181021,TOPUP,EUR,18.69,FAILED,NaN,NaN,NaN,NaN,INBOUND,user_6926,2019-05-01 09:30:06.606771
681022,transaction_2181022,TRANSFER,EUR,1.70,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_1011,2019-04-30 10:10:36.810961
681023,transaction_2181023,TRANSFER,EUR,4.26,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_3170,2019-05-01 11:34:25.078035
681024,transaction_2181024,TRANSFER,EUR,1.70,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_8195,2019-04-30 16:51:26.445105


In [56]:
def first6(s):
    return s[:7] 

In [57]:
transactions['created_date2'] = transactions['created_date'].apply(first6)

In [58]:
transactions

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date,created_date2
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_4368,2018-04-03 03:34:21.784487,2018-04
1,transaction_1,CARD_PAYMENT,AED,15.50,COMPLETED,FALSE,4111.0,Dubai,ARE,OUTBOUND,user_2355,2019-03-19 06:15:59.537032,2019-03
2,transaction_2,CARD_PAYMENT,AED,43.40,COMPLETED,FALSE,5814.0,Dubai,ARE,OUTBOUND,user_2355,2019-03-18 18:53:41.323032,2019-03
3,transaction_3,TRANSFER,AED,10043.01,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_2355,2019-03-22 14:20:01.513032,2019-03
4,transaction_4,CARD_PAYMENT,AED,43.81,COMPLETED,FALSE,5651.0,Abu Dhabi,ARE,OUTBOUND,user_14318,2019-03-22 05:42:50.316652,2019-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
681021,transaction_2181021,TOPUP,EUR,18.69,FAILED,NaN,NaN,NaN,NaN,INBOUND,user_6926,2019-05-01 09:30:06.606771,2019-05
681022,transaction_2181022,TRANSFER,EUR,1.70,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_1011,2019-04-30 10:10:36.810961,2019-04
681023,transaction_2181023,TRANSFER,EUR,4.26,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_3170,2019-05-01 11:34:25.078035,2019-05
681024,transaction_2181024,TRANSFER,EUR,1.70,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_8195,2019-04-30 16:51:26.445105,2019-04


In [59]:
pd.crosstab(index=transactions['created_date2'], columns=transactions['user_id'])

user_id,user_0,user_1,user_10,user_100,user_1000,user_10000,user_10001,user_10002,user_10003,user_10004,...,user_9990,user_9991,user_9992,user_9993,user_9994,user_9995,user_9996,user_9997,user_9998,user_9999
created_date2,,,,,,,,,,,,,,,,,,,,,
2018-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-02,0,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
2018-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-05,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-06,0,4,5,0,0,0,0,0,6,0,...,0,2,3,0,0,4,0,0,0,0
2018-07,0,4,0,0,0,0,0,0,6,0,...,0,0,14,40,0,0,0,0,0,0
2018-08,0,4,26,0,0,0,2,0,14,0,...,0,0,14,46,0,0,0,5,0,0
2018-09,29,4,2,0,0,0,0,0,0,0,...,0,0,0,42,0,0,0,47,0,0


In [60]:
transactions_last_15_days =pd.crosstab(index=transactions['created_date2'], columns=transactions['user_id']).iloc[16:]

In [61]:
transactions_last_15_days

user_id,user_0,user_1,user_10,user_100,user_1000,user_10000,user_10001,user_10002,user_10003,user_10004,...,user_9990,user_9991,user_9992,user_9993,user_9994,user_9995,user_9996,user_9997,user_9998,user_9999
created_date2,,,,,,,,,,,,,,,,,,,,,
2019-05,18,2,11,0,1,26,0,0,1,0,...,0,0,0,6,7,1,0,0,0,10


In [62]:
transactions_last_15_days= transactions_last_15_days.T

In [63]:
transactions_last_15_days

created_date2,2019-05
user_id,
user_0,18
user_1,2
user_10,11
user_100,0
user_1000,1
...,...
user_9995,1
user_9996,0
user_9997,0


In [64]:
transactions_last_15_days= transactions_last_15_days.rename({'2019-05':'transactions_last_15_days'}, axis =1)

In [65]:
transactions_last_15_days

created_date2,transactions_last_15_days
user_id,
user_0,18
user_1,2
user_10,11
user_100,0
user_1000,1
...,...
user_9995,1
user_9996,0
user_9997,0


In [66]:
users =pd.merge(users,transactions_last_15_days, how='left',left_on='user_id',right_index=True)

In [67]:
users.head(5)

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,brand,number_of_transactions,completed_transactions,transactions_last_15_days
0,user_0,13,1,1,0,0,3,0,1,0,1,545.0,520.0,18.0
1,user_1,12,0,1,1,0,75,1,0,0,1,146.0,133.0,2.0
2,user_2,14,0,1,0,0,71,1,0,0,1,282.0,268.0,0.0
3,user_3,11,1,1,0,0,219,1,0,0,1,158.0,140.0,7.0
4,user_4,13,1,1,0,0,0,0,1,0,1,59.0,50.0,0.0


In [68]:
pd.crosstab(index=users['transactions_last_15_days'], columns=users['plan'])

plan,0,1
transactions_last_15_days,,
0.0,7720,204
1.0,894,84
2.0,703,66
3.0,553,37
4.0,450,38
...,...,...
166.0,0,1
171.0,0,1
180.0,0,1


# 2.3.4 Amount_usd

In [69]:
transactions[['user_id','amount_usd']].groupby(['user_id']).sum()

,amount_usd
user_id,
user_0,7122.21
user_1,14162.08
user_10,6256.46
user_100,6344.10
user_1000,901.62
...,...
user_9995,324.41
user_9996,1710.40
user_9997,15043.07


In [70]:
total_amount_usd =transactions[['user_id','amount_usd']].groupby(['user_id']).sum()
total_amount_usd =total_amount_usd.rename({'amount_usd': 'total_amount_usd'}, axis=1)

In [71]:
total_amount_usd

,total_amount_usd
user_id,
user_0,7122.21
user_1,14162.08
user_10,6256.46
user_100,6344.10
user_1000,901.62
...,...
user_9995,324.41
user_9996,1710.40
user_9997,15043.07


In [72]:
users =pd.merge(users,total_amount_usd, how='left',left_on='user_id',right_index=True)

In [73]:
users.head(2)

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,brand,number_of_transactions,completed_transactions,transactions_last_15_days,total_amount_usd
0,user_0,13,1,1,0,0,3,0,1,0,1,545.0,520.0,18.0,7122.21
1,user_1,12,0,1,1,0,75,1,0,0,1,146.0,133.0,2.0,14162.08


# 2.3.5 Amount usd completed

In [74]:
transactions[transactions['transactions_state'] == 'COMPLETED'].groupby('user_id')['amount_usd'].sum()

user_id
user_0        6925.08
user_1       12955.54
user_10       5438.92
user_100      4230.10
user_1000      843.12
               ...   
user_9995      239.85
user_9996     1681.76
user_9997     7650.53
user_9998      520.72
user_9999        8.51
Name: amount_usd, Length: 14821, dtype: float64

In [75]:
total_amount_usd_completed =transactions[transactions['transactions_state'] == 'COMPLETED'].groupby('user_id')['amount_usd'].sum()

In [76]:
total_amount_usd_completed = pd.DataFrame({'total_amount_usd_completed': total_amount_usd_completed})

In [77]:
users =pd.merge(users,total_amount_usd_completed, how='left',left_on='user_id',right_index=True)

In [78]:
users

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,brand,number_of_transactions,completed_transactions,transactions_last_15_days,total_amount_usd,total_amount_usd_completed
0,user_0,13,1,1,0,0,3,0,1,0,1,545.0,520.0,18.0,7122.21,6925.08
1,user_1,12,0,1,1,0,75,1,0,0,1,146.0,133.0,2.0,14162.08,12955.54
2,user_2,14,0,1,0,0,71,1,0,0,1,282.0,268.0,0.0,2983.40,2723.90
3,user_3,11,1,1,0,0,219,1,0,0,1,158.0,140.0,7.0,8270.65,6999.29
4,user_4,13,1,1,0,0,0,0,1,0,1,59.0,50.0,0.0,1068.40,981.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15539,user_15539,14,0,2,0,0,0,1,0,0,0,134.0,91.0,0.0,6428.68,4274.95
15540,user_15540,12,0,2,0,0,0,1,0,0,1,16.0,10.0,0.0,103.68,97.10
15541,user_15541,10,1,2,0,0,16,0,1,0,0,1115.0,1062.0,53.0,19560.56,17721.66
15542,user_15542,10,1,1,0,1,44,1,0,0,1,30.0,23.0,3.0,443.23,408.72


In [79]:
users.isna().any()

user_id                          False
birth_year                       False
city                             False
created_date                     False
user_settings_crypto_unlocked    False
plan                             False
num_contacts                     False
North-East                       False
UK                               False
West-South                       False
brand                            False
number_of_transactions            True
completed_transactions            True
transactions_last_15_days         True
total_amount_usd                  True
total_amount_usd_completed        True
dtype: bool

In [80]:
users['number_of_transactions'] = users['number_of_transactions'].fillna(0)
users['completed_transactions'] = users['completed_transactions'].fillna(0)
users['transactions_last_15_days']=users['transactions_last_15_days'].fillna(0)
users['total_amount_usd'] = users['total_amount_usd'].fillna(0)
users['total_amount_usd_completed']=users['total_amount_usd_completed'].fillna(0)

# 2.4 Notifications

In [81]:
notif.head(3)

,reason,channel,status,user_id,created_date
0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1252,2018-12-02 17:58:33.320645
1,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_12783,2018-12-01 23:09:37.367127
2,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_10244,2018-12-04 02:57:56.425660


# 2.4.1 Channel

In [82]:
pd.crosstab(index=notif['channel'], columns=notif['user_id']).T

channel,EMAIL,PUSH,SMS
user_id,,,
user_0,6,2,0
user_1,10,5,0
user_10,5,3,0
user_100,2,3,0
user_1000,1,0,0
...,...,...,...
user_9995,2,0,0
user_9996,1,2,0
user_9997,2,5,1


In [83]:
x =pd.crosstab(index=notif['channel'], columns=notif['user_id']).T

In [84]:
(x > 0).astype(int)

channel,EMAIL,PUSH,SMS
user_id,,,
user_0,1,1,0
user_1,1,1,0
user_10,1,1,0
user_100,1,1,0
user_1000,1,0,0
...,...,...,...
user_9995,1,0,0
user_9996,1,1,0
user_9997,1,1,1


In [85]:
y =(x > 0).astype(int)

In [86]:
users =pd.merge(users,y, how='left',left_on='user_id',right_index=True)

In [87]:
users

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,brand,number_of_transactions,completed_transactions,transactions_last_15_days,total_amount_usd,total_amount_usd_completed,EMAIL,PUSH,SMS
0,user_0,13,1,1,0,0,3,0,1,0,1,545.0,520.0,18.0,7122.21,6925.08,1.0,1.0,0.0
1,user_1,12,0,1,1,0,75,1,0,0,1,146.0,133.0,2.0,14162.08,12955.54,1.0,1.0,0.0
2,user_2,14,0,1,0,0,71,1,0,0,1,282.0,268.0,0.0,2983.40,2723.90,1.0,1.0,0.0
3,user_3,11,1,1,0,0,219,1,0,0,1,158.0,140.0,7.0,8270.65,6999.29,1.0,1.0,0.0
4,user_4,13,1,1,0,0,0,0,1,0,1,59.0,50.0,0.0,1068.40,981.43,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15539,user_15539,14,0,2,0,0,0,1,0,0,0,134.0,91.0,0.0,6428.68,4274.95,1.0,1.0,0.0
15540,user_15540,12,0,2,0,0,0,1,0,0,1,16.0,10.0,0.0,103.68,97.10,1.0,1.0,0.0
15541,user_15541,10,1,2,0,0,16,0,1,0,0,1115.0,1062.0,53.0,19560.56,17721.66,1.0,1.0,0.0
15542,user_15542,10,1,1,0,1,44,1,0,0,1,30.0,23.0,3.0,443.23,408.72,1.0,1.0,0.0


In [88]:
users.isna().any()

user_id                          False
birth_year                       False
city                             False
created_date                     False
user_settings_crypto_unlocked    False
plan                             False
num_contacts                     False
North-East                       False
UK                               False
West-South                       False
brand                            False
number_of_transactions           False
completed_transactions           False
transactions_last_15_days        False
total_amount_usd                 False
total_amount_usd_completed       False
EMAIL                             True
PUSH                              True
SMS                               True
dtype: bool

In [89]:
users['EMAIL'] = users['EMAIL'].fillna(0)
users['PUSH'] = users['PUSH'].fillna(0)
users['SMS'] = users['SMS'].fillna(0)

In [90]:
pd.crosstab(index=notif['channel'], columns=users['plan'])

plan,0,1
channel,,
EMAIL,2426,214
PUSH,10967,854
SMS,1001,82


# 2.4.2  Status

In [91]:
notif.head(2)

,reason,channel,status,user_id,created_date
0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1252,2018-12-02 17:58:33.320645
1,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_12783,2018-12-01 23:09:37.367127


In [92]:
notif1=notif[notif['channel']=='PUSH']

In [93]:
notif2=notif[notif['channel']=='EMAIL']

In [94]:
notif3=notif[notif['channel']=='SMS']

In [95]:
n1 = notif1[notif1['status'] == 'SENT'].groupby('user_id')['channel'].count()

In [96]:
n2 = notif2[notif2['status'] == 'SENT'].groupby('user_id')['channel'].count()

In [97]:
n3 = notif3[notif3['status'] == 'SENT'].groupby('user_id')['channel'].count()

In [98]:
n1.to_frame()

,channel
user_id,
user_0,2
user_1,5
user_10,3
user_100,3
user_10000,5
...,...
user_9992,1
user_9993,3
user_9994,3


In [99]:
n2.to_frame()

,channel
user_id,
user_0,6
user_1,5
user_10,5
user_100,2
user_1000,1
...,...
user_9995,1
user_9996,1
user_9997,2


In [100]:
n3.to_frame()

,channel
user_id,
user_1001,1
user_10132,1
user_10177,1
user_10309,1
user_10354,1
...,...
user_9728,4
user_9847,1
user_9870,4


In [101]:
users =pd.merge(users,n1, how='left',left_on='user_id',right_index=True)

In [102]:
users =pd.merge(users,n2, how='left',left_on='user_id',right_index=True)

In [103]:
users =pd.merge(users,n3, how='left',left_on='user_id',right_index=True)

In [104]:
users = users.rename({'channel_x':"PUSH_sent",'channel_y':'EMAIL_sent','channel':'SMS_sent'},axis=1)

In [105]:
users.isna().any()

user_id                          False
birth_year                       False
city                             False
created_date                     False
user_settings_crypto_unlocked    False
plan                             False
num_contacts                     False
North-East                       False
UK                               False
West-South                       False
brand                            False
number_of_transactions           False
completed_transactions           False
transactions_last_15_days        False
total_amount_usd                 False
total_amount_usd_completed       False
EMAIL                            False
PUSH                             False
SMS                              False
PUSH_sent                         True
EMAIL_sent                        True
SMS_sent                          True
dtype: bool

In [106]:
users['PUSH_sent']=users['PUSH_sent'].fillna(0)
users['EMAIL_sent']=users['EMAIL_sent'].fillna(0)
users['SMS_sent']=users['SMS_sent'].fillna(0)

In [107]:
users.isna().any()

user_id                          False
birth_year                       False
city                             False
created_date                     False
user_settings_crypto_unlocked    False
plan                             False
num_contacts                     False
North-East                       False
UK                               False
West-South                       False
brand                            False
number_of_transactions           False
completed_transactions           False
transactions_last_15_days        False
total_amount_usd                 False
total_amount_usd_completed       False
EMAIL                            False
PUSH                             False
SMS                              False
PUSH_sent                        False
EMAIL_sent                       False
SMS_sent                         False
dtype: bool

In [108]:
users['PUSH_sent']=users['PUSH_sent'].fillna(0)
users['EMAIL_sent']=users['EMAIL_sent'].fillna(0)
users['SMS_sent']=users['SMS_sent'].fillna(0)

In [109]:
users.head()

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,...,completed_transactions,transactions_last_15_days,total_amount_usd,total_amount_usd_completed,EMAIL,PUSH,SMS,PUSH_sent,EMAIL_sent,SMS_sent
0,user_0,13,1,1,0,0,3,0,1,0,...,520.0,18.0,7122.21,6925.08,1.0,1.0,0.0,2.0,6.0,0.0
1,user_1,12,0,1,1,0,75,1,0,0,...,133.0,2.0,14162.08,12955.54,1.0,1.0,0.0,5.0,5.0,0.0
2,user_2,14,0,1,0,0,71,1,0,0,...,268.0,0.0,2983.40,2723.90,1.0,1.0,0.0,3.0,2.0,0.0
3,user_3,11,1,1,0,0,219,1,0,0,...,140.0,7.0,8270.65,6999.29,1.0,1.0,0.0,6.0,0.0,0.0
4,user_4,13,1,1,0,0,0,0,1,0,...,50.0,0.0,1068.40,981.43,1.0,1.0,0.0,4.0,3.0,0.0


In [110]:
users['Perc_comp_transactions'] = (users['completed_transactions']/users['number_of_transactions'])*100

In [111]:
users['Perc_compl_amount'] = (users['total_amount_usd_completed']/users['total_amount_usd'])*100

In [112]:
users.isna().any()

user_id                          False
birth_year                       False
city                             False
created_date                     False
user_settings_crypto_unlocked    False
plan                             False
num_contacts                     False
North-East                       False
UK                               False
West-South                       False
brand                            False
number_of_transactions           False
completed_transactions           False
transactions_last_15_days        False
total_amount_usd                 False
total_amount_usd_completed       False
EMAIL                            False
PUSH                             False
SMS                              False
PUSH_sent                        False
EMAIL_sent                       False
SMS_sent                         False
Perc_comp_transactions            True
Perc_compl_amount                 True
dtype: bool

In [113]:
users['Perc_comp_transactions'] = users['Perc_comp_transactions'].fillna(0)
users['Perc_compl_amount'] = users['Perc_compl_amount'].fillna(0)

In [114]:
users.head(10)

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,...,total_amount_usd,total_amount_usd_completed,EMAIL,PUSH,SMS,PUSH_sent,EMAIL_sent,SMS_sent,Perc_comp_transactions,Perc_compl_amount
0,user_0,13,1,1,0,0,3,0,1,0,...,7122.21,6925.08,1.0,1.0,0.0,2.0,6.0,0.0,95.412844,97.232179
1,user_1,12,0,1,1,0,75,1,0,0,...,14162.08,12955.54,1.0,1.0,0.0,5.0,5.0,0.0,91.095890,91.480489
2,user_2,14,0,1,0,0,71,1,0,0,...,2983.40,2723.90,1.0,1.0,0.0,3.0,2.0,0.0,95.035461,91.301870
3,user_3,11,1,1,0,0,219,1,0,0,...,8270.65,6999.29,1.0,1.0,0.0,6.0,0.0,0.0,88.607595,84.628052
4,user_4,13,1,1,0,0,0,0,1,0,...,1068.40,981.43,1.0,1.0,0.0,4.0,3.0,0.0,84.745763,91.859790
5,user_5,13,0,2,0,0,0,0,1,0,...,41.00,40.00,1.0,1.0,0.0,0.0,0.0,0.0,66.666667,97.560976
6,user_6,14,0,2,1,0,77,0,0,1,...,1451.88,1379.17,1.0,1.0,0.0,2.0,2.0,0.0,89.411765,94.992010
7,user_7,13,0,2,0,0,10,0,0,1,...,50680.06,30699.52,1.0,1.0,0.0,3.0,4.0,0.0,79.442149,60.575145
8,user_8,13,0,2,0,0,42,1,0,0,...,6897.18,6210.27,1.0,0.0,0.0,0.0,3.0,0.0,92.842105,90.040712
9,user_9,12,0,1,1,0,3,0,1,0,...,3512.15,3185.00,1.0,1.0,0.0,7.0,5.0,0.0,92.537313,90.685193


# 2.4.3 April 19

In [115]:
transactions_april_2019 =pd.crosstab(index=transactions['created_date2'], columns=transactions['user_id']).iloc[15:16]

In [116]:
transactions_april_2019

user_id,user_0,user_1,user_10,user_100,user_1000,user_10000,user_10001,user_10002,user_10003,user_10004,...,user_9990,user_9991,user_9992,user_9993,user_9994,user_9995,user_9996,user_9997,user_9998,user_9999
created_date2,,,,,,,,,,,,,,,,,,,,,
2019-04,74,31,14,0,37,18,0,0,0,0,...,0,0,0,15,57,3,0,38,0,0


In [117]:
transactions_april_2019= transactions_april_2019.T

In [118]:
transactions_april_2019= transactions_april_2019.rename({'2019-04':'transactions_april_2019'}, axis =1)

In [119]:
users =pd.merge(users,transactions_april_2019, how='left',left_on='user_id',right_index=True)

# 2.4.4 Jan_Feb_Mar_2019

In [120]:
transactions_Jan_Feb_Mar_2019 =pd.crosstab(index=transactions['created_date2'], columns=transactions['user_id']).iloc[12:15]

In [121]:
transactions_Jan_Feb_Mar_2019

user_id,user_0,user_1,user_10,user_100,user_1000,user_10000,user_10001,user_10002,user_10003,user_10004,...,user_9990,user_9991,user_9992,user_9993,user_9994,user_9995,user_9996,user_9997,user_9998,user_9999
created_date2,,,,,,,,,,,,,,,,,,,,,
2019-01,61,25,5,42,0,3,0,0,0,1,...,0,0,10,8,0,0,9,18,3,0
2019-02,6,23,8,8,0,0,0,8,0,0,...,0,0,0,5,0,0,0,8,26,0
2019-03,11,10,5,0,9,48,0,0,0,0,...,5,0,2,16,0,0,0,2,7,0


In [122]:
transactions_Jan_Feb_Mar_2019=transactions_Jan_Feb_Mar_2019.sum()

In [123]:
transactions_Jan_Feb_Mar_2019= transactions_Jan_Feb_Mar_2019.T

In [124]:
transactions_Jan_Feb_Mar_2019

user_id
user_0       78
user_1       58
user_10      18
user_100     50
user_1000     9
             ..
user_9995     0
user_9996     9
user_9997    28
user_9998    36
user_9999     0
Length: 15012, dtype: int64

In [125]:
transactions_Jan_Feb_Mar_2019 = pd.DataFrame({'transactions_Jan_Feb_Mar_2019': transactions_Jan_Feb_Mar_2019})

In [126]:
users =pd.merge(users,transactions_Jan_Feb_Mar_2019, how='left',left_on='user_id',right_index=True)

In [127]:
users

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,...,EMAIL,PUSH,SMS,PUSH_sent,EMAIL_sent,SMS_sent,Perc_comp_transactions,Perc_compl_amount,transactions_april_2019,transactions_Jan_Feb_Mar_2019
0,user_0,13,1,1,0,0,3,0,1,0,...,1.0,1.0,0.0,2.0,6.0,0.0,95.412844,97.232179,74.0,78.0
1,user_1,12,0,1,1,0,75,1,0,0,...,1.0,1.0,0.0,5.0,5.0,0.0,91.095890,91.480489,31.0,58.0
2,user_2,14,0,1,0,0,71,1,0,0,...,1.0,1.0,0.0,3.0,2.0,0.0,95.035461,91.301870,21.0,257.0
3,user_3,11,1,1,0,0,219,1,0,0,...,1.0,1.0,0.0,6.0,0.0,0.0,88.607595,84.628052,22.0,53.0
4,user_4,13,1,1,0,0,0,0,1,0,...,1.0,1.0,0.0,4.0,3.0,0.0,84.745763,91.859790,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15539,user_15539,14,0,2,0,0,0,1,0,0,...,1.0,1.0,0.0,1.0,2.0,0.0,67.910448,66.498099,17.0,16.0
15540,user_15540,12,0,2,0,0,0,1,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,62.500000,93.653549,0.0,0.0
15541,user_15541,10,1,2,0,0,16,0,1,0,...,1.0,1.0,0.0,1.0,3.0,0.0,95.246637,90.598940,115.0,589.0
15542,user_15542,10,1,1,0,1,44,1,0,0,...,1.0,1.0,0.0,1.0,2.0,0.0,76.666667,92.213975,3.0,24.0


# 2.5 Amount_usd max min mean

In [128]:
transactions[transactions['transactions_state'] == 'COMPLETED'].groupby('user_id')['amount_usd'].max()

user_id
user_0        510.64
user_1        851.06
user_10       255.22
user_100      382.98
user_1000     238.31
              ...   
user_9995      82.96
user_9996     500.00
user_9997    1000.00
user_9998      79.53
user_9999       8.51
Name: amount_usd, Length: 14821, dtype: float64

In [129]:
max_amount_usd_completed =transactions[transactions['transactions_state'] == 'COMPLETED'].groupby('user_id')['amount_usd'].max()

In [130]:
max_amount_usd_completed = pd.DataFrame({'max_amount_usd_completed': max_amount_usd_completed})

In [131]:
users =pd.merge(users,max_amount_usd_completed, how='left',left_on='user_id',right_index=True)

In [132]:
min_amount_usd_completed =transactions[transactions['transactions_state'] == 'COMPLETED'].groupby('user_id')['amount_usd'].min()

In [133]:
min_amount_usd_completed = pd.DataFrame({'min_amount_usd_completed': min_amount_usd_completed})

In [134]:
users =pd.merge(users,min_amount_usd_completed, how='left',left_on='user_id',right_index=True)

In [135]:
mean_amount_usd_completed =transactions[transactions['transactions_state'] == 'COMPLETED'].groupby('user_id')['amount_usd'].mean()

In [136]:
mean_amount_usd_completed = pd.DataFrame({'mean_amount_usd_completed': mean_amount_usd_completed})

In [137]:
users =pd.merge(users,mean_amount_usd_completed, how='left',left_on='user_id',right_index=True)

In [138]:
users

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,...,PUSH_sent,EMAIL_sent,SMS_sent,Perc_comp_transactions,Perc_compl_amount,transactions_april_2019,transactions_Jan_Feb_Mar_2019,max_amount_usd_completed,min_amount_usd_completed,mean_amount_usd_completed
0,user_0,13,1,1,0,0,3,0,1,0,...,2.0,6.0,0.0,95.412844,97.232179,74.0,78.0,510.64,0.58,13.317462
1,user_1,12,0,1,1,0,75,1,0,0,...,5.0,5.0,0.0,91.095890,91.480489,31.0,58.0,851.06,0.94,97.410075
2,user_2,14,0,1,0,0,71,1,0,0,...,3.0,2.0,0.0,95.035461,91.301870,21.0,257.0,194.38,0.04,10.163806
3,user_3,11,1,1,0,0,219,1,0,0,...,6.0,0.0,0.0,88.607595,84.628052,22.0,53.0,510.64,1.87,49.994929
4,user_4,13,1,1,0,0,0,0,1,0,...,4.0,3.0,0.0,84.745763,91.859790,0.0,4.0,200.00,1.50,19.628600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15539,user_15539,14,0,2,0,0,0,1,0,0,...,1.0,2.0,0.0,67.910448,66.498099,17.0,16.0,297.65,0.05,46.977473
15540,user_15540,12,0,2,0,0,0,1,0,0,...,0.0,0.0,0.0,62.500000,93.653549,0.0,0.0,43.84,0.46,9.710000
15541,user_15541,10,1,2,0,0,16,0,1,0,...,1.0,3.0,0.0,95.246637,90.598940,115.0,589.0,1100.00,0.01,16.687062
15542,user_15542,10,1,1,0,1,44,1,0,0,...,1.0,2.0,0.0,76.666667,92.213975,3.0,24.0,85.11,0.01,17.770435


In [139]:
users.isna().any()

user_id                          False
birth_year                       False
city                             False
created_date                     False
user_settings_crypto_unlocked    False
plan                             False
num_contacts                     False
North-East                       False
UK                               False
West-South                       False
brand                            False
number_of_transactions           False
completed_transactions           False
transactions_last_15_days        False
total_amount_usd                 False
total_amount_usd_completed       False
EMAIL                            False
PUSH                             False
SMS                              False
PUSH_sent                        False
EMAIL_sent                       False
SMS_sent                         False
Perc_comp_transactions           False
Perc_compl_amount                False
transactions_april_2019           True
transactions_Jan_Feb_Mar_

In [140]:
users['transactions_april_2019']= users['transactions_april_2019'].fillna(0) 
users['transactions_Jan_Feb_Mar_2019']= users['transactions_Jan_Feb_Mar_2019'].fillna(0)  
users['max_amount_usd_completed']= users['max_amount_usd_completed'].fillna(0)           
users['min_amount_usd_completed']= users['min_amount_usd_completed'].fillna(0)          
users['mean_amount_usd_completed']= users['mean_amount_usd_completed'].fillna(0)

In [141]:
users

,user_id,birth_year,city,created_date,user_settings_crypto_unlocked,plan,num_contacts,North-East,UK,West-South,...,PUSH_sent,EMAIL_sent,SMS_sent,Perc_comp_transactions,Perc_compl_amount,transactions_april_2019,transactions_Jan_Feb_Mar_2019,max_amount_usd_completed,min_amount_usd_completed,mean_amount_usd_completed
0,user_0,13,1,1,0,0,3,0,1,0,...,2.0,6.0,0.0,95.412844,97.232179,74.0,78.0,510.64,0.58,13.317462
1,user_1,12,0,1,1,0,75,1,0,0,...,5.0,5.0,0.0,91.095890,91.480489,31.0,58.0,851.06,0.94,97.410075
2,user_2,14,0,1,0,0,71,1,0,0,...,3.0,2.0,0.0,95.035461,91.301870,21.0,257.0,194.38,0.04,10.163806
3,user_3,11,1,1,0,0,219,1,0,0,...,6.0,0.0,0.0,88.607595,84.628052,22.0,53.0,510.64,1.87,49.994929
4,user_4,13,1,1,0,0,0,0,1,0,...,4.0,3.0,0.0,84.745763,91.859790,0.0,4.0,200.00,1.50,19.628600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15539,user_15539,14,0,2,0,0,0,1,0,0,...,1.0,2.0,0.0,67.910448,66.498099,17.0,16.0,297.65,0.05,46.977473
15540,user_15540,12,0,2,0,0,0,1,0,0,...,0.0,0.0,0.0,62.500000,93.653549,0.0,0.0,43.84,0.46,9.710000
15541,user_15541,10,1,2,0,0,16,0,1,0,...,1.0,3.0,0.0,95.246637,90.598940,115.0,589.0,1100.00,0.01,16.687062
15542,user_15542,10,1,1,0,1,44,1,0,0,...,1.0,2.0,0.0,76.666667,92.213975,3.0,24.0,85.11,0.01,17.770435
